In [74]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [75]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [76]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [77]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///./Resources/hawaii.sqlite")

In [78]:
# Declare a Base using `automap_base()`
Base = automap_base()
# reflect an existing database into a new model
Base.prepare(engine, reflect=True)
# reflect the tables


In [79]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [80]:
# Save references to each table
Measurement=Base.classes.measurement
Station=Base.classes.station

In [81]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [82]:
first_row_measurement = session.query(Measurement).first()
first_row_measurement.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1f26e53a640>,
 'tobs': 65.0,
 'station': 'USC00519397',
 'id': 1,
 'prcp': 0.08,
 'date': '2010-01-01'}

In [83]:
first_row_station = session.query(Station).first()
first_row_station.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1f26e53ad00>,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397',
 'elevation': 3.0,
 'latitude': 21.2716,
 'id': 1}

# Exploratory Precipitation Analysis

In [84]:
# Find the most recent date in the data set.
latest_measurement = session.query(Measurement).order_by(Measurement.date.desc()).first()
latest_measurement = latest_measurement.__dict__
latest_measurement_date=latest_measurement['date']
print(latest_measurement_date)

2017-08-23


In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
x = latest_measurement_date.split("-")
#print(int(x[0])-1)
query_date=str(int(x[0])-1) + '-' + x[1] + '-' + x[2]
print(query_date)

# Perform a query to retrieve the data and precipitation scores
results=session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date > query_date).\
    order_by(Measurement.date).all()
# Save the query results as a Pandas DataFrame and set the index to the date column
date = [results[0] for results in results]
prcp = [results[1] for results in results]
precip_dict={'date': date, 'prcp': prcp}
precip_df=pd.DataFrame(data=precip_dict)
# Set "None" values as 0
precip_df['prcp'] = precip_df['prcp'].fillna(0)
#precip_df.head()

# Sort the dataframe by date
precip_df=precip_df.sort_values(by=['date'])

# Use Pandas Plotting with Matplotlib to plot the data
x_axis = np.arange(len(precip_df))
tick_locations = [value for value in x_axis]
#plt.figure(figsize=(20,3))
plt.bar(x_axis, precip_df["prcp"], color='r', align="center")
plt.xticks(tick_locations, precip_df["date"], rotation="vertical")
plt.show()


In [113]:
#print(prcp)

In [119]:
# Use Pandas to calcualte the summary statistics for the precipitation data
precip_df.describe()

,prcp
count,2223.000000
mean,0.159951
std,0.441220
min,0.000000
25%,0.000000
50%,0.010000
75%,0.110000
max,6.700000


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()